In [ ]:
import os
import shutil
import cclib


outscandir1='./out_run/C271_hexil/'
outscandir=outscandir1+'homo_lumo/out_dd/'

w=[]
z=[]
y=[]

count=0
correction_lumo=[]
correction_homo=[]
for root, dirs, filenames in os.walk(outscandir):
    for m in dirs:
        for root1, dirs1, filenames1 in os.walk(outscandir+m+'/'):
            for n in filenames1:
                mol=cclib.parser.QChem(outscandir+m+'/'+n)                
                data=mol.parse()
   
                print(m,n)
                homo=data.moenergies[0][data.homos[0]]
                
                homo_menos=data.moenergies[0][data.homos[0]-1]
                lumo=data.moenergies[0][data.homos[0]+1]
                lumo_mas=data.moenergies[0][data.homos[0]+2]
                
                correction_lumo.append(lumo)
                correction_homo.append(homo)
                
                
                
                
                
                
                
    print(len(correction_lumo))            
    promedio_lumo=sum(correction_lumo)/len(correction_lumo)
    promedio_homo=sum(correction_homo)/len(correction_homo)
    
    ## all
    promedio_homo=0.9248*promedio_homo-0.8084-promedio_homo
    promedio_lumo=0.9248*promedio_lumo-0.8084-promedio_lumo
    
    
    ###4-3ht and 4-T
    
    #promedio_homo=0.9248*promedio_homo-0.8084-promedio_homo+0.3525
    #promedio_lumo=0.9248*promedio_lumo-0.8084-promedio_lumo-0.5965
    
    for m in dirs:
        for root1, dirs1, filenames1 in os.walk(outscandir+m+'/'):
            for n in filenames1:
                mol=cclib.parser.QChem(outscandir+m+'/'+n)                
                data=mol.parse()   
                
                homo=data.moenergies[0][data.homos[0]]
                lumo=data.moenergies[0][data.homos[0]+1]
                lumo=lumo+promedio_lumo
                homo=homo+promedio_homo
                
                y.append(lumo)
                w.append(homo)
                z.append(lumo-homo) 
                
                
                
                
            
                count=count+1
escribe=open(outscandir+'homo_lumo.txt','w')
escribe.write('#     HOMO(eV)           LUMO(eV)               gap(eV)      '+'\n')
for i,j in enumerate(w):
    escribe.write("{:<15s}  {:<15s}  {:<15s} ".format(str(round(w[i],4)),str(round(y[i],4)),str(round(z[i],4)))+'\n')
escribe.close()
    



In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.mlab as mlab
%matplotlib inline
import numpy as np
import pandas as pd
from scipy import stats, integrate
from scipy.interpolate import spline


print(int(21*abs(max(w)-min(w))))
print(int(21*abs(max(y)-min(y))))

fig = plt.gcf()
fig, ax1 = plt.subplots()



SS =plt.hist(w,normed=True,bins=int(21*abs(max(w)-min(w))),range=(min(w), max(w)),alpha=0.5,facecolor='green', rwidth=0.81, color='b', histtype='barstacked')
new_SS=[]
new2_SS=[]


new_SS.append(0)
new2_SS.append(SS[1][0]-(-SS[1][0]+SS[1][1])/2)
for index,i in enumerate(SS[0]):
    new_SS.append(i)
    new2_SS.append(SS[1][index]+(-SS[1][0]+SS[1][1])/2)
new_SS.append(0)
new2_SS.append(SS[1][-1]+(-SS[1][0]+SS[1][1])/2)






DD=plt.hist(y,normed=True,bins=int(21*abs(max(y)-min(y))),range=(min(y), max(y)),alpha=0.5,facecolor='blue', rwidth=0.81, color='b', histtype='barstacked')

new_DD=[]
new2_DD=[]


new_DD.append(0)
new2_DD.append(DD[1][0]-(-DD[1][0]+DD[1][1])/2)
for index,i in enumerate(DD[0]):
    new_DD.append(i)
    new2_DD.append(DD[1][index]+(-DD[1][0]+DD[1][1])/2)
new_DD.append(0)
new2_DD.append(DD[1][-1]+(-DD[1][0]+DD[1][1])/2)



xnew1=np.linspace(min(new2_DD),max(new2_DD),100)
smooth1 = spline(new2_DD,new_DD,xnew1)
plt.plot(xnew1,smooth1,'k--',linewidth=1.)


xnew2=np.linspace(min(new2_SS),max(new2_SS),100)
smooth2 = spline(new2_SS,new_SS,xnew2)
plt.plot(xnew2,smooth2,'k--',linewidth=1.)

########
CC=[]
FF=[]
index1=[]
index2=[]
for i,j in enumerate(smooth2):
    if j==max(smooth2):
        xmax=xnew2[i]
        
        
for index,i in enumerate(xnew2):
        if (xnew2[index]-xmax)>0:
            CC.append(float(np.absolute(smooth2[index]-0.65)+np.absolute(i-xmax)))
            index1.append(index)
        elif (xnew2[index]-xmax) < 0:
            FF.append(float(np.absolute(smooth2[index]-0.65)+np.absolute(i-xmax)))
            index2.append(index)
CC=[CC,index1]
FF=[FF,index2]

for i,j in enumerate(CC[0]):
    if j==min(CC[0]):
        index1=CC[1][i]

for i,j in enumerate(FF[0]):
    if j==min(FF[0]):
        index2=FF[1][i]
#########

std_homo=[]
for index,i in enumerate(smooth2):
    if i >=max(smooth2)/2:
        std_homo.append(xnew2[index])
print(max(std_homo)-min(std_homo))

CC=[]
for index,i in enumerate(xnew2):
    if xnew2[index1]>= i >=xnew2[index2]:
        CC.append(smooth2[index])

media_homo=[]
for index,i in enumerate(xnew2):
    if xnew2[index1]>= i >=xnew2[index2]:
        media_homo.append(i*smooth2[index]/sum(CC))
print(sum(media_homo))


########
#########
#######

CC=[]
FF=[]
index1=[]
index2=[]
for i,j in enumerate(smooth1):
    if j==max(smooth1):
        xmax=xnew1[i]
        
        
for index,i in enumerate(xnew1):
        if (xnew1[index]-xmax)>0:
            CC.append(float(np.absolute(smooth1[index]-0.65)+np.absolute(i-xmax)))
            index1.append(index)
        elif (xnew1[index]-xmax) < 0:
            FF.append(float(np.absolute(smooth1[index]-0.65)+np.absolute(i-xmax)))
            index2.append(index)
CC=[CC,index1]
FF=[FF,index2]

for i,j in enumerate(CC[0]):
    if j==min(CC[0]):
        index1=CC[1][i]

for i,j in enumerate(FF[0]):
    if j==min(FF[0]):
        index2=FF[1][i]

std_lumo=[]
for index,i in enumerate(smooth1):
    if i >=max(smooth1)/2:
        std_lumo.append(xnew1[index])
print(max(std_lumo)-min(std_lumo))

CC=[]
for index,i in enumerate(xnew1):
    if xnew1[index1]>= i >=xnew1[index2]:
        CC.append(smooth1[index])

media_lumo=[]
for index,i in enumerate(xnew1):
    if xnew1[index1]>= i >=xnew1[index2]:
        media_lumo.append(i*smooth1[index]/sum(CC))
print(sum(media_lumo))



plt.xlabel('eV', fontsize=14, fontweight='bold')
plt.ylabel('Freq. norm.', fontsize=14, fontweight='bold')
#################################################3

plt.axis([-6.5, -1.0, 0., 7] )

plt.xticks(size=14)
plt.yticks(size=14)



ax2 = ax1.twinx()

ax2.twiny()

ZZ=plt.hist(z,normed=True,bins=int(20*abs(max(z)-min(z))),range=(min(z), max(z)),alpha=0.5,facecolor='red',rwidth=0.81, color='b', histtype='barstacked')

new_ZZ=[]
new2_ZZ=[]

new_ZZ.append(0)
new2_ZZ.append(ZZ[1][0]-(-ZZ[1][0]+ZZ[1][1])/2)
for index,i in enumerate(ZZ[0]):
    new_ZZ.append(i)
    new2_ZZ.append(ZZ[1][index]+(-ZZ[1][0]+ZZ[1][1])/2)
new_ZZ.append(0)
new2_ZZ.append(ZZ[1][-1]+(-ZZ[1][0]+ZZ[1][1])/2)


xnew3=np.linspace(min(new2_ZZ),max(new2_ZZ),100)
smooth3 = spline(new2_ZZ,new_ZZ,xnew3)


CC=[]
FF=[]
index1=[]
index2=[]
for i,j in enumerate(smooth3):
    if j==max(smooth3):
        xmax=xnew3[i]
        
        
for index,i in enumerate(xnew3):
        if (xnew3[index]-xmax)>0:
            CC.append(float(np.absolute(smooth3[index]-0.65)+np.absolute(i-xmax)))
            index1.append(index)
        elif (xnew3[index]-xmax) < 0:
            FF.append(float(np.absolute(smooth3[index]-0.65)+np.absolute(i-xmax)))
            index2.append(index)
CC=[CC,index1]
FF=[FF,index2]

for i,j in enumerate(CC[0]):
    if j==min(CC[0]):
        index1=CC[1][i]

for i,j in enumerate(FF[0]):
    if j==min(FF[0]):
        index2=FF[1][i]



std_gap=[]
for index,i in enumerate(smooth3):
    if i >=max(smooth3)/2:
        std_gap.append(xnew3[index])
print(max(std_gap)-min(std_gap))

CC=[]
for index,i in enumerate(xnew3):
    if xnew3[index1]>= i >=xnew3[index2]:
        CC.append(smooth3[index])

media_gap=[]
for index,i in enumerate(xnew3):
    if xnew3[index1]>= i >=xnew3[index2]:
        media_gap.append(i*smooth3[index]/sum(CC))
print(sum(media_gap))



plt.plot(xnew3,smooth3,'k--',linewidth=1.)

plt.axis([0., 5, 0., 7])

plt.xticks(size=14)
plt.yticks(size=14)



ax2.tick_params('both',labelsize=14)
fig.tight_layout()
ax2.invert_yaxis()





plt.savefig(outscandir+'HOMO_LUMOS_sc.png', bbox_inches="tight",dpi=200)

plt.show()

